In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import optuna
from catboost import CatBoostClassifier as CBC
from lightgbm import LGBMClassifier as LGBM
from sklearn.preprocessing import OrdinalEncoder as OE
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv')
train.shape, test.shape

((140700, 20), (93800, 19))

In [3]:
train.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [4]:
test.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
1,140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,2.0,NaN,NaN,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
2,140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,NaN,4.0,NaN,NaN,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
3,140703,Nalini,Female,23.0,Rajkot,Student,NaN,5.0,NaN,6.84,1.0,NaN,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No
4,140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,NaN,5.0,NaN,NaN,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Profession                             104070 non-null  object 
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93800 entries, 0 to 93799
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     93800 non-null  int64  
 1   Name                                   93800 non-null  object 
 2   Gender                                 93800 non-null  object 
 3   Age                                    93800 non-null  float64
 4   City                                   93800 non-null  object 
 5   Working Professional or Student        93800 non-null  object 
 6   Profession                             69168 non-null  object 
 7   Academic Pressure                      18767 non-null  float64
 8   Work Pressure                          75022 non-null  float64
 9   CGPA                                   18766 non-null  float64
 10  Study Satisfaction                     18767 non-null  float64
 11  Jo

In [7]:
trc = list(train.columns)
tec = list(test.columns)
NUM_COLS, CAT_COLS = [],[]
for i in trc:
    if i not in tec:
        TARGET = i
    elif train[i].dtype in [int,float]:
        NUM_COLS.append(i)
    else:
        CAT_COLS.append(i)

print((len(CAT_COLS) + len(NUM_COLS) + 1) == len(trc))
NUM_COLS.remove('id')

True


In [8]:
oe = OE(handle_unknown = 'use_encoded_value', unknown_value=-1, encoded_missing_value=-2)
oe.fit(train[CAT_COLS])
train[CAT_COLS] = oe.transform(train[CAT_COLS])
test[CAT_COLS] = oe.transform(test[CAT_COLS])

In [9]:
for i in CAT_COLS:
    train[i] = train[i].astype(np.int64)    
    test[i] = test[i].astype(np.int64)
    print(f"{i}: ",train[i].isna().sum(),test[i].isna().sum())

Name:  0 0
Gender:  0 0
City:  0 0
Working Professional or Student:  0 0
Profession:  0 0
Sleep Duration:  0 0
Dietary Habits:  0 0
Degree:  0 0
Have you ever had suicidal thoughts ?:  0 0
Family History of Mental Illness:  0 0


In [10]:
# for i in NUM_COLS:
#     if train[i].isna().sum() >0 :
#         train[i] = train[i].fillna(np.mean(train[i]))
#         test[i] = test[i].fillna(np.mean(test[i]))                           
#         print(f"{i}: ",train[i].isna().sum(),test[i].isna().sum())

In [11]:
train.info()
#great, encoding fixed all by 6 columns of nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  int64  
 2   Gender                                 140700 non-null  int64  
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  int64  
 5   Working Professional or Student        140700 non-null  int64  
 6   Profession                             140700 non-null  int64  
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

In [12]:
test.info() #even better, only 5 test columns now have nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93800 entries, 0 to 93799
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     93800 non-null  int64  
 1   Name                                   93800 non-null  int64  
 2   Gender                                 93800 non-null  int64  
 3   Age                                    93800 non-null  float64
 4   City                                   93800 non-null  int64  
 5   Working Professional or Student        93800 non-null  int64  
 6   Profession                             93800 non-null  int64  
 7   Academic Pressure                      18767 non-null  float64
 8   Work Pressure                          75022 non-null  float64
 9   CGPA                                   18766 non-null  float64
 10  Study Satisfaction                     18767 non-null  float64
 11  Jo

In [13]:
BOTH = CAT_COLS + NUM_COLS

In [14]:
trainsmall = train.sample(51000)
trainsmall.reset_index(inplace=True,drop=True)
trainsmall.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,78693,58,1,49.0,69,1,20,NaN,1.0,NaN,NaN,1.0,15,7,81,0,12.0,5.0,0,0
1,82468,188,0,19.0,94,1,-2,NaN,4.0,NaN,NaN,1.0,19,20,45,1,4.0,5.0,0,1
2,119108,421,0,46.0,17,1,55,NaN,3.0,NaN,NaN,2.0,15,20,81,0,1.0,3.0,0,0
3,82627,177,0,51.0,78,1,33,NaN,5.0,NaN,NaN,3.0,19,20,63,1,6.0,5.0,1,0
4,96239,114,0,23.0,94,0,-2,5.0,NaN,8.62,3.0,NaN,15,15,80,1,10.0,1.0,0,1


In [15]:
oo=0

def optim(trial):
    cb_params = {
        'objective': 'CrossEntropy',
        'iterations': 1512, 
        'learning_rate': trial.suggest_float('learning_rate',0.01,0.12),
        'depth': trial.suggest_int('depth',2,12),
        #'subsample':trial.suggest_float('subsample',0,1),
        'reg_lambda': trial.suggest_int('reg_lambda',0,30),
        #'bagging_temperature':trial.suggest_float('bagging_temperature',1,2),
        'min_data_in_leaf':20,
        'use_best_model':True,
        'task_type':"GPU",
        
    }
    FOLDS = 3
    scores = np.zeros(FOLDS)
    kf = KFold(n_splits = FOLDS)
    for i, (trndex,valdex) in enumerate(kf.split(trainsmall.id)):
        xtrain,ytrain = trainsmall.loc[trndex,BOTH], trainsmall.loc[trndex,TARGET]
        xval, yval    = trainsmall.loc[valdex,BOTH], trainsmall.loc[valdex,TARGET]
        model = CBC(cat_features = CAT_COLS, **cb_params)
        model.fit(xtrain,ytrain,
                 eval_set=(xval,yval),
                 verbose=False,
                 early_stopping_rounds = 25,
                 use_best_model=True)
        
        ypred =  model.predict(xval) 
        ascore = accuracy_score(yval,ypred)
        scores[i] = ascore
    
    global oo
    print(f"{oo}: {np.mean(scores)}") 
    oo +=1
    return np.mean(scores)
study = optuna.create_study(direction='maximize')
study.optimize(optim,n_trials = 200)

#report the results 
print('Best parameters:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2024-11-05 19:35:03,458] A new study created in memory with name: no-name-a22a6441-ba2c-40e7-95ae-84e165de7510
[I 2024-11-05 19:42:10,830] Trial 0 finished with value: 0.9392352941176471 and parameters: {'learning_rate': 0.05409479929538654, 'depth': 10, 'reg_lambda': 13}. Best is trial 0 with value: 0.9392352941176471.


0: 0.9392352941176471


[I 2024-11-05 19:42:43,496] Trial 1 finished with value: 0.9393333333333334 and parameters: {'learning_rate': 0.11298563255511176, 'depth': 4, 'reg_lambda': 27}. Best is trial 1 with value: 0.9393333333333334.


1: 0.9393333333333334


[I 2024-11-05 19:43:22,696] Trial 2 finished with value: 0.9390980392156862 and parameters: {'learning_rate': 0.11496064855751756, 'depth': 6, 'reg_lambda': 12}. Best is trial 1 with value: 0.9393333333333334.


...

197: 0.9393333333333332


[I 2024-11-06 06:57:01,194] Trial 198 finished with value: 0.9391372549019609 and parameters: {'learning_rate': 0.05585301701413598, 'depth': 6, 'reg_lambda': 7}. Best is trial 155 with value: 0.9398039215686275.


198: 0.9391372549019609


[I 2024-11-06 06:58:05,612] Trial 199 finished with value: 0.9393529411764705 and parameters: {'learning_rate': 0.05105664443715378, 'depth': 5, 'reg_lambda': 6}. Best is trial 155 with value: 0.9398039215686275.


199: 0.9393529411764705
Best parameters: {'learning_rate': 0.055905048327263715, 'depth': 5, 'reg_lambda': 12}
Best score: 0.9398039215686275
